# SpeakUp2.0: Neon Eyetrackign LSL with Pupil Cloud Alignment 

### Info Documents 
Location Repository
Github Repository 
Jupyter Notebook

see documentation here: https://pupil-invisible-lsl-relay.readthedocs.io/en/stable/guides/time_alignment.html




### Requirements
Please install the necessary packages in requirements.txt using pip install -r requirements.txt

### 0. Importing Necessary Packages

In [26]:
import json
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import os  
import subprocess  # to run the command line tool
import tkinter # GUI toolkit to open and save files
from tkinter import filedialog # GUI toolkit to open and save files
import re
#import liblsl
#from lsl_relay import time_alignment


### 1. Defyining Relevant Directories, Variables and Functions

In [34]:
## DIRECTORIES
input_folder_xdf = r'\\fileserver.dccn.nl\project\3025011.01\pilots\pilot_data_davide'
input_folder_cloud_exports = os.path.abspath('./cloud_exports/') 
output_aligned_folder = os.path.abspath('./aligned_cloud_lsl/')

print('Will loud xdf files from folder: ', input_folder_xdf)
print('Will loud cloud export files from folder: ', input_folder_cloud_exports)
print('Will save aligned cloud export files to folder: ', output_aligned_folder)

## PARAMETERS
xdf_file_keyword= "speech" # Change this to the recording to the keyword (e.g., "speech", "baseline")
print('Will load xdf files with keyword: ', xdf_file_keyword)
cloud_folder_keyword = "d" # Change this to the recording to the keyword 
print('Will load cloud export folder starting with keyword: ', cloud_folder_keyword)

xdf_cloud_patter_matching =  re.compile(r'pilot_(\d+)_d(\d+)')
print('Will use the following pattern to match xdf and cloud export files: ', xdf_cloud_patter_matching)

data_of_interest = 'gaze.csv' # This is the cloud data file that will contain the LSL time stamps as a column
column_of_interest = "timestamp [ns]" # The data of interest should contain a column with this name 

## FUNCTIONS
# Define a simple linear mapping function
def perform_linear_mapping(input_data, parameters):
    return parameters["intercept"] + input_data * parameters["slope"]

print('Function to perform linear mapping defined')


Will loud xdf files from folder:  \\fileserver.dccn.nl\project\3025011.01\pilots\pilot_data_davide
Will loud cloud export files from folder:  f:\SpeakUp-2.0\2_PREPROCESSING\3_PupilCloud_Sync\cloud_exports
Will save aligned cloud export files to folder:  f:\SpeakUp-2.0\2_PREPROCESSING\3_PupilCloud_Sync\aligned_cloud_lsl
Will load xdf files with keyword:  speech
Will load cloud export folder starting with keyword:  d
Will use the following pattern to match xdf and cloud export files:  re.compile('pilot_(\\d+)_d(\\d+)')
Function to perform linear mapping defined


### 2. Identifying Relevant XDF files and Cloud Epxort folders in based on input directories 

In [28]:
## XDF FILES
xdf_files = []  # Initialize an empty list to store paths of XDF files

try:
    if os.path.exists(input_folder_xdf):  # Check if the input folder exists
       
        # Traverse through the directory and its subdirectories to find XDF files
        for root, dirs, files in os.walk(input_folder_xdf):  # Walk through the directory tree
          
            for file in files:
                
                if file.endswith(".xdf") and xdf_file_keyword in file:  # Check if the file is an XDF file and that it contains the word "speech" in its name
                    
                    xdf_files.append(os.path.join(root, file))  # Append the full path to the xdf_files list
                    
        if xdf_files:  # Check if any XDF files were found
            print(f'We have identified the following XDF files: {xdf_files}')
        
        else:  # If no XDF files were found
            print('No XDF files found.')
            
    else: # If the input folder does not exist or is not accessible
        print(f'The folder {input_folder_xdf} does not exist or is not accessible.')

except Exception as e:
    print(f"An error occurred: {str(e)}")



## CLOUD EXPORT FILES
cloud_export_folders = []  # Initialize an empty list to store paths of cloud export files
try:
    if os.path.exists(input_folder_cloud_exports):  # Check if the input folder exists
       
        # Traverse through the directory and its subdirectories to find XDF files
        for root, dirs, files in os.walk(input_folder_cloud_exports):  # Walk through the directory tree
          
            for subdir in dirs:
                
                # Check if the folder is a 'dx' folder (e.g., dx)
                if subdir.startswith(cloud_folder_keyword) and subdir[1:].isdigit():  # This checks if the folder is named dx (e.g., d1, d2)                    
                    full_path = os.path.join(root, subdir)
                    cloud_export_folders.append(full_path)  # Append the full path of the folder to the list


        if cloud_export_folders:  # Check if any folders were found
            print(f'We have identified the following cloud export folders: {cloud_export_folders}')
    
        else:  # If no folders were found
            print('No cloud export folders found.')
            
    else:  # If the input folder does not exist or is not accessible
        print(f'The folder {input_folder_cloud_exports} does not exist or is not accessible.')

except Exception as e:
    print(f"An error occurred: {str(e)}")

We have identified the following XDF files: ['\\\\fileserver.dccn.nl\\project\\3025011.01\\pilots\\pilot_data_davide\\pilot_01\\day2\\pilot_01_d2_speech_lsl.xdf', '\\\\fileserver.dccn.nl\\project\\3025011.01\\pilots\\pilot_data_davide\\pilot_01\\day3\\pilot_01_d3_speech_lsl.xdf', '\\\\fileserver.dccn.nl\\project\\3025011.01\\pilots\\pilot_data_davide\\pilot_02\\day2\\pilot_02_d2_speech_lsl.xdf', '\\\\fileserver.dccn.nl\\project\\3025011.01\\pilots\\pilot_data_davide\\pilot_02\\day3\\pilot_02_d3_speech_lsl.xdf', '\\\\fileserver.dccn.nl\\project\\3025011.01\\pilots\\pilot_data_davide\\pilot_03\\day2\\pilot_03_d2_speech_lsl.xdf', '\\\\fileserver.dccn.nl\\project\\3025011.01\\pilots\\pilot_data_davide\\pilot_04\\day2\\pilot_04_d2_speech_lsl.xdf', '\\\\fileserver.dccn.nl\\project\\3025011.01\\pilots\\pilot_data_davide\\pilot_05\\day2\\pilot_05_d2_speech_lsl.xdf', '\\\\fileserver.dccn.nl\\project\\3025011.01\\pilots\\pilot_data_davide\\pilot_06\\day2\\pilot_06_d2_speech_lsl.xdf']
We have ide

### 2a. Alternatively, the User selects their own XDF files and correspdonding Cloud Folder (Only 1 pair per time)

In [7]:
## XDF FILE
root = tkinter.Tk()
root.attributes('-topmost',True)
root.iconify()

xdf_files = filedialog.askopenfilename(title="Select One XDF file", filetypes=[("XDF Files", "*.xdf")])

root.destroy()

print('You have selected the following XDF files: ' + str(xdf_files))

## CLOUD FOLDER
root = tkinter.Tk()
root.attributes('-topmost',True)
root.iconify()

cloud_folder = filedialog.askdirectory(title="Select One corresponding cloud folder")

root.destroy()


print('You have selected the following cloud folder: ' + str(cloud_folder))

You have selected the following XDF files: //fileserver.dccn.nl/project/3025011.01/pilots/pilot_data_davide/pilot_01/day2/pilot_01_d2_speech_lsl.xdf


### 3. Now we run the 'lsl_relay_time_alignment' with subprocess in a loop 

In [31]:
for xdf_file in xdf_files:
    print(f'Looking into the XDF file: {xdf_file}')
    
    # Extract relevant key words from file name to match later with cloud export folder
    fnam = os.path.basename(xdf_file)[:-4]    # Extract the file name from the path and assings it to fnam, whilst removing the '.xdf' extension (i.e., the last 4 characters in the string)
    participant_num = '_'.join(fnam.split('_')[0:2])  # Extract the participant number from the file name by splitting the string at the underscores and selecting the first two elements
    day_num = fnam.split('_')[2]  # Extract the day number from the file name by splitting the string at the underscores and selecting the third element


    # Extrac the identifier from the xdf file name
    xdf_match = xdf_cloud_patter_matching.search(fnam)
    if not xdf_match :
        # print(f'No matching for video: {fnam}')
        continue  # Skip current iteration if the pattern is not found
    xdf_identifier = xdf_match.group(0)  # Extract the matched portion of the string from the regex result.


    # Loop over the cloud export folders to find the corresponding folder
    for cloud_export_folder in cloud_export_folders:
         
        normalized_path = os.path.normpath(cloud_export_folder) # Normalize the path to ensure proper handling of backslashes and forward slashes
        path_parts = normalized_path.split(os.sep)   # Split the normalized path into parts based on the OS-specific separator (e.g., '\' on Windows and '/' on Linux/Mac)
        # Extract relevant key words from file name to match later with cloud export folder
        
        participant_num = path_parts[-2]  # !! MAKE SURE THIS IS THE CORRECT INDEX !! Extract the participant number from the second to last part of the path
        day_num = path_parts[-1]        # !! MAKE SURE THIS IS THE CORRECT INDEX !! Extract the day from the last part of the path

        temp_cloud_name = f'{participant_num}_{day_num}'  # Create a temporary cloud name based on the extracted participant number and day number

        # Extrac the identifier from the cloud export folder name
        cloud_match = xdf_cloud_patter_matching.search(temp_cloud_name)
        if not cloud_match :
            # print(f'No matching for video: {fnam}')
            continue
        cloud_identifier = cloud_match.group(0)


        # Check if the identifiers match
        if xdf_identifier == cloud_identifier:
            print(f'Found matching cloud export folder: {cloud_export_folder}')
            

            # Now we run the time alignment 
            # Define the command and the arguments
            command = ['lsl_relay_time_alignment', xdf_file, cloud_export_folder]

            result = subprocess.run(command, capture_output=True, text=True)

            #The post-hoc time alignment outputs a json file ‘time_alignment_parameters.json’ in the same directory where the events.csv file from the pupil cloud export is located. 

            # Print the output
            print(result.stdout)
            print(result.stderr)

        else: # If no matching is found
            print(f'No matching cloud export found for xdf file : {os.path.basename(xdf_file)}')


Looking into the XDF file: \\fileserver.dccn.nl\project\3025011.01\pilots\pilot_data_davide\pilot_01\day2\pilot_01_d2_speech_lsl.xdf
Looking into pilot_01_d2
Found matching cloud export folder: f:\SpeakUp-2.0\2_PREPROCESSING\3_PupilCloud_Sync\cloud_exports\pilot_01\d2
[10/04/24 16:17:18] INFO     Saving logs to ./time_sync_posthoc.log  cli.py:189
                    INFO     Loading XDF events from  xdf_cloud_time_sync.py:54
                             \\fileserver.dccn.nl\pro                          
                             ject\3025011.01\pilots\p                          
                             ilot_data_davide\pilot_0                          
                             1\day2\pilot_01_d2_speec                          
                             h_lsl.xdf                                         
                    INFO     Importing XDF file                    pyxdf.py:202
                             \\fileserver.dccn.nl\project\3025011.             
           

### 4. Now we run the Linear Mapping to Add LSL times to our Cloud Data.csv

In [39]:
for cloud_export_folder in cloud_export_folders:
    print(f'Looking into the cloud export folder: {cloud_export_folder}')

    #Extract info about the key names (p number and day) from the cloud export folder
    normalized_path = os.path.normpath(cloud_export_folder) # Normalize the path to ensure proper handling of backslashes and forward slashes
    path_parts = normalized_path.split(os.sep)   # Split the normalized path into parts based on the OS-specific separator (e.g., '\' on Windows and '/' on Linux/Mac)
    
    participant_num = path_parts[-2]  # !! MAKE SURE THIS IS THE CORRECT INDEX !! Extract the participant number from the second to last part of the path
    day_num = path_parts[-1]        # !! MAKE SURE THIS IS THE CORRECT INDEX !! Extract the day from the last part of the path

    # load the data of interest
    print(f'Loading the data of interest: {data_of_interest}')
    data = pd.read_csv(os.path.join(cloud_export_folder, data_of_interest))
    
    # Extract the timestamp column from the gaze data
    column_timestamp =  data[column_of_interest]

    # Convert nanoseconds to seconds and create a new column "timestamp [s]"
    data["timestamp [s]"] = column_timestamp * 1e-9

    # Load the time alignment parameters from the json file
    print(f'Loading time alignment parameters from: {os.path.join(cloud_export_folder, "time_alignment_parameters.json")}')
    with open(os.path.abspath(os.path.join(cloud_export_folder, "time_alignment_parameters.json"))) as file:
        parameter_dict = json.load(file)


    # Apply the linear model to transform timestamps from Pupil Cloud to LSL time domain
    print('Applying linear model to transform timestamps from Pupil Cloud to LSL time domain')
    data["lsl_time [s]"] = perform_linear_mapping(
        data["timestamp [s]"], parameter_dict["cloud_to_lsl"]
    )

    # Save the time-aligned data as a CSV file in the appropriate folder

     # Define the output file path and save the time-aligned data
    output_file_path = os.path.join(
        os.path.abspath(output_aligned_folder), 
        f'{participant_num}_{day_num}_cloud_lsl_aligned_{data_of_interest}'
    )
    #Save csv data
    data.to_csv(output_file_path, index=False)

    print(f'Time-aligned data saved to: {output_file_path}')

print('Time alignment completed. Llok into your folder', output_aligned_folder)


Looking into the cloud export folder: f:\SpeakUp-2.0\2_PREPROCESSING\3_PupilCloud_Sync\cloud_exports\pilot_01\d2
Loading the data of interest: gaze.csv
Loading time alignment parameters from: f:\SpeakUp-2.0\2_PREPROCESSING\3_PupilCloud_Sync\cloud_exports\pilot_01\d2\time_alignment_parameters.json
Applying linear model to transform timestamps from Pupil Cloud to LSL time domain
Looking into the cloud export folder: f:\SpeakUp-2.0\2_PREPROCESSING\3_PupilCloud_Sync\cloud_exports\pilot_01\d3
Loading the data of interest: gaze.csv
Loading time alignment parameters from: f:\SpeakUp-2.0\2_PREPROCESSING\3_PupilCloud_Sync\cloud_exports\pilot_01\d3\time_alignment_parameters.json
Applying linear model to transform timestamps from Pupil Cloud to LSL time domain
Looking into the cloud export folder: f:\SpeakUp-2.0\2_PREPROCESSING\3_PupilCloud_Sync\cloud_exports\pilot_02\d2
Loading the data of interest: gaze.csv
Loading time alignment parameters from: f:\SpeakUp-2.0\2_PREPROCESSING\3_PupilCloud_Sync